### GPT-J (6B)

In [1]:
from pathlib import Path
import argparse

from transformers import (
    GPT2Config,
    GPT2Tokenizer,
    GPT2DoubleHeadsModel,
    GPT2LMHeadModel,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    GPTJForCausalLM
)

import os
from scipy.special import softmax
import torch
import numpy as np
from tqdm import tqdm

/home/cgagne/cvar_generation/conda_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
sys.path.append('../')

In [3]:
from generator import generate

In [4]:
modelname= '../models/pretrained/EleutherAI-gpt-j-6B'
device = 'cpu'

In [5]:
model = GPTJForCausalLM.from_pretrained(modelname, torch_dtype=torch.float16).to(device)
tokenizer = AutoTokenizer.from_pretrained(modelname)

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): Embedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): GPTJMLP(
          (fc_in): Linear(in_features=4096, out_features=16384, bias=True)
          (fc_out): Linear(in_features=16384, out_features=4096, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
      (1): GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=

In [7]:
prompt = 'I have errands today. I need to go to the doctors.'
inputs = tokenizer(prompt, return_tensors='pt').to(device)
inputs

{'input_ids': tensor([[   40,   423, 11454,  1746,  1909,    13,   314,   761,   284,   467,
           284,   262,  7519,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [8]:
output, _ = generate(model, 
                    tokenizer,
                    input_ids=inputs['input_ids'],
                    attention_mask=inputs['attention_mask'],
                    max_length=inputs['input_ids'].shape[1]+20, 
                    num_beams = 1,
                    temperature=1, 
                    num_return_sequences=10,
                    do_sample=True, 
                    eos_token_id=13,
                    top_k = 10, 
                    top_p=0.95,
                    allowed_word_ids = None,
                            )

RuntimeError: "LayerNormKernelImpl" not implemented for 'Half'

In [ ]:
tokenizer.batch_decode(output, skip_special_tokens=True)

In [9]:
output2 = model.generate(input_ids=inputs['input_ids'],
               attention_mask=inputs['attention_mask'],
               max_length=inputs['input_ids'].shape[1]+20, 
               num_beams = 1,
               temperature=1, 
               num_return_sequences=10,
               do_sample=True, 
               eos_token_id=13,
               top_k = 10, 
               top_p=0.95,
               allowed_word_ids = None)

Setting `pad_token_id` to `eos_token_id`:13 for open-end generation.


RuntimeError: "LayerNormKernelImpl" not implemented for 'Half'

In [ ]:
tokenizer.batch_decode(output2)